In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim


from torch_geometric.nn import GCNConv
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures


# print(-torch.log10(torch.tensor(0.)))
print(torch.log(torch.tensor(0.235)))
print(torch.log(torch.tensor(0.62)) + torch.log(torch.tensor(0.55)) + torch.log(torch.tensor(0.69)))  


tensor(-1.4482)
tensor(-1.4469)


In [57]:

class Agent(nn.Module):
    def __init__(self, num_node_features,hidden_channels1,hidden_channels2):
        super().__init__()
        self.conv1 = GCNConv(num_node_features, hidden_channels1)
        self.conv2 = GCNConv( hidden_channels1,hidden_channels2)
        self.lin1 = nn.Linear(hidden_channels2, 64)
        self.lin2 = nn.Linear(64, 10)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index  # x: [2708, 1433]

        x = self.conv1(x, edge_index)            # [2708, hidden1]
        x = F.relu(x)
        x = self.conv2(x, edge_index)            # [2708, hidden2]
        x = F.relu(x)

        x = self.lin1(x)                          # [2708, 64]
        x = F.relu(x)
        x = self.lin2(x)                          # [2708, fc_out]

        x = F.tanh(x)
        return x


In [58]:

dataset = Planetoid(root='data/Planetoid', name='Cora', transform=NormalizeFeatures())
dataset[0]


Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

In [59]:
dataset[0].edge_index[:,7]

tensor([332,   2])

In [60]:

model = Agent(num_node_features=dataset.num_node_features,hidden_channels1=16,hidden_channels2=32)
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
data = dataset[0]  
out = model(data)
o = out[1]
mapped = torch.where(o > 0.5, 1,
           torch.where(o < -0.5, -1, 0))

mapped

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [61]:
import pandas as pd

In [62]:
df = pd.read_csv('./sim/monitor/logs/packets_log.csv')
df

,Node,Packet,Direction,Protocol,Port,Time,Size,Offset,src IP,dest IP,prev_hop,next_hop,total_hops
0,6,0,TX,UDP,520,0.048468,52,0,192.166.1.104,224.0.0.9,Null,Null,0
1,6,0,TX,UDP,520,0.048468,52,0,192.166.1.122,224.0.0.9,Null,Null,0
2,6,0,TX,UDP,520,0.048468,52,0,192.166.1.139,224.0.0.9,Null,Null,0
3,6,0,TX,UDP,520,0.048468,52,0,192.166.1.141,224.0.0.9,Null,Null,0
4,6,0,TX,UDP,520,0.048468,52,0,192.166.1.143,224.0.0.9,Null,Null,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
90447,3,1809,RX,UDP,520,99.621200,212,0,192.166.1.55,224.0.0.9,Null,Null,0
90448,6,1812,RX,UDP,520,99.621200,212,0,192.166.1.61,224.0.0.9,Null,Null,0
90449,4,1817,RX,UDP,520,99.621200,212,0,192.166.1.57,224.0.0.9,Null,Null,0
90450,0,1818,RX,UDP,520,99.622600,212,0,192.166.1.4,224.0.0.9,Null,Null,0


In [63]:
valid_packets = df[(df["Port"] == 9) | (df["Port"] == 49153)]
valid_packets

,Node,Packet,Direction,Protocol,Port,Time,Size,Offset,src IP,dest IP,prev_hop,next_hop,total_hops
88063,5,1026,RX,UDP,9,10.0018,1052,0,1.1.10.1,2.2.10.1,Client,2,2
88064,5,1026,TX,UDP,9,10.0018,1052,0,1.1.10.1,2.2.10.1,Client,2,2
88065,3,1028,RX,UDP,9,10.0018,1052,0,1.1.12.1,2.2.12.1,Client,11,2
88066,3,1028,TX,UDP,9,10.0018,1052,0,1.1.12.1,2.2.12.1,Client,11,2
88067,12,1027,RX,UDP,9,10.0027,1052,0,1.1.11.1,2.2.11.1,Client,7,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
89731,5,1437,TX,UDP,49153,39.0244,1052,0,2.2.10.1,1.1.10.1,2,Client,2
89732,2,1441,RX,UDP,49153,39.0310,1052,0,2.2.10.1,1.1.14.1,Server,6,2
89733,2,1441,TX,UDP,49153,39.0310,1052,0,2.2.10.1,1.1.14.1,Server,6,2
89734,6,1441,RX,UDP,49153,39.0337,1052,0,2.2.10.1,1.1.14.1,2,Client,2


In [71]:
valid_packets[valid_packets['Packet']==1026]

,Node,Packet,Direction,Protocol,Port,Time,Size,Offset,src IP,dest IP,prev_hop,next_hop,total_hops
88063,5,1026,RX,UDP,9,10.0018,1052,0,1.1.10.1,2.2.10.1,Client,2,2
88064,5,1026,TX,UDP,9,10.0018,1052,0,1.1.10.1,2.2.10.1,Client,2,2
88073,2,1026,RX,UDP,9,10.0037,1052,0,1.1.10.1,2.2.10.1,5,Server,2
88074,2,1026,TX,UDP,9,10.0037,1052,0,1.1.10.1,2.2.10.1,5,Server,2
88095,2,1026,RX,UDP,49153,10.0226,1052,0,2.2.10.1,1.1.10.1,Server,5,2
88096,2,1026,TX,UDP,49153,10.0226,1052,0,2.2.10.1,1.1.10.1,Server,5,2
88097,5,1026,RX,UDP,49153,10.0244,1052,0,2.2.10.1,1.1.10.1,2,Client,2
88098,5,1026,TX,UDP,49153,10.0244,1052,0,2.2.10.1,1.1.10.1,2,Client,2


In [77]:
packet_summary = valid_packets.groupby(
    ["Packet", "Port", "total_hops"]).size().reset_index(name="count")

packet_summary["status"] = packet_summary.apply(
    lambda row: "OK" if row["count"] % (2 * row['total_hops'])==0 else "LOST", axis=1
)
lost_packets = packet_summary[packet_summary['status'] == 'LOST']


/var/folders/07/z4c9jvjx1bvgvh3zx2djp3rh0000gn/T/ipykernel_62609/2908239613.py:5: RuntimeWarning: divide by zero encountered in scalar remainder
  lambda row: "OK" if row["count"] % (2 * row['total_hops'])==0 else "LOST", axis=1


In [80]:
valid_packets = valid_packets.merge(
    packet_summary[['Packet', 'Port', 'total_hops', 'status']],
    on=['Packet', 'Port', 'total_hops'],
    how='left'
)

In [84]:
valid_packets[valid_packets['Packet'] == 1113]

,Node,Packet,Direction,Protocol,Port,Time,Size,Offset,src IP,dest IP,prev_hop,next_hop,total_hops,status
84,5,1113,RX,UDP,9,12.0018,1052,0,1.1.10.1,2.2.10.1,Client,2,2,LOST
85,5,1113,TX,UDP,9,12.0018,1052,0,1.1.10.1,2.2.10.1,Client,2,2,LOST


In [ ]:
tx_packets = valid_packets[valid_packets["Direction"] == "TX"]
rx_packets = valid_packets[valid_packets["Direction"] == "RX"]